In [ ]:
!pip install langchain faiss-cpu tiktoken openai pypdf

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent, Tool
from langchain.tools import HumanInputRun
from langchain.agents import AgentType

In [ ]:
openai_api_key = ""

llm = ChatOpenAI(temperature=0.0, openai_api_key=openai_api_key)
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [3]:
def save_pdf_to_faiss_db(pdf_file, save_dir):
    # Loading PDF document
    loader = PyPDFLoader(pdf_file)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    print(f'Number of documents: {len(texts)}')
    print(texts[0])

    # Get FAISS embedding and save
    db = FAISS.from_documents(texts, embeddings)
    db.save_local(save_dir)
    return db

def create_retrival_chain(db):
    # Creating retrieval QA chains
    db_retriever = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=db.as_retriever()
    )
    return db_retriever

def create_retrieval_tool(pdf_file, save_dir, tool_name, tool_description):

    db = save_pdf_to_faiss_db(pdf_file, save_dir)
    db_retriever = create_retrival_chain(db)

    retrieval_tool = Tool(
            name=tool_name,
            func=db_retriever.run,
            description=tool_description,
            return_direct=True
        )
    return retrieval_tool

def get_input() -> str:
    print("Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.")
    contents = []
    while True:
        try:
            line = input()
        except EOFError:
            break
        if line == "q":
            break
        contents.append(line)
    return "\n".join(contents)

In [6]:
file1 = "./data/住院兒童發生跌倒原因.pdf"
savedir1 = "faiss_idx_childfall"
tool_name1 = "Children fall QA system"
tool_description1 = "用以回答住院兒童跌倒相關問題。"

childfall_qa_tool = create_retrieval_tool(pdf_file=file1, save_dir=savedir1, tool_name=tool_name1, tool_description=tool_description1)

Number of documents: 4
page_content='住院兒童發生跌倒原因  \n \n兒童因素 :  \n \n1.因疾病造成意識不清、躁動、虛弱、步  \n \n  態不穩  \n \n2.活動力旺盛、喜歡爬上爬下  \n \n照顧者因素 : \n \n1.雙側床欄沒有完全拉高  \n \n2.讓兒在陪客床上活動  \n \n3.使用嬰兒推車或輪椅沒有正確使用煞  \n \n  車、安全帶  \n \n4.照顧者離開，兒童沒有人看顧  \n \n藥物因素 : \n \n1.鎮靜麻醉藥 :嗜睡、眩暈、注意力不集中  \n \n2.利尿劑、軟便劑 :增加上廁所次數  \n \n3.降血糖藥 :暈眩、低血糖  \n \n環境因素 : \n \n1.地板濕滑  \n \n2.照明不足  \n \n3.輪椅、推車不足' metadata={'source': './data/住院兒童發生跌倒原因.pdf', 'page': 0}


In [7]:
file2 = "./data/孕產婦住院預防跌倒須知.pdf"
savedir2 = "faiss_idx_pregnantfall"
tool_name2 = "Pregnant women fall QA system"
tool_description2 = "用以回答住院孕婦跌倒相關問題。"

pwomenfall_qa_tool = create_retrieval_tool(pdf_file=file2, save_dir=savedir2, tool_name=tool_name2, tool_description=tool_description2)

Number of documents: 2
page_content='孕產婦住院預防跌倒須知  \n婦產科衛教護理師  陳翠玲  整理 \n一、常見發生跌倒原因：  \n1.個人健康因素：頭暈、貧血、高血壓、低血糖、長期臥床安胎。  \n2.環境因素：地面濕滑、照明不足等。  \n3.藥物因素：  \n(1)軟便劑：如廁次數增加、急忙如廁。  \n(2)降血壓藥：姿態性低血壓、暈眩、肌肉無力、步態不穩。  \n(3)降血糖藥：暈眩、低血糖症狀。  \n(4)利尿劑：如廁次數增加、姿態性低血壓。  \n(5)抗癲癇藥與精神科用藥：嗜睡、暈眩、反應時間延遲、注意力不集中。  \n(6)使用減痛分娩致下肢無力。  \n二、跌倒常見併發症？  \n孕產婦跌倒可能造成嚴重的外部傷害 (如 圖一)，增加失能機會；而對於寶寶則增加早產或死產的風險。  \n三、如何預防因服用藥物而跌倒？  \n1.服藥期間有任何問題，請先與醫師討論，勿自行調整用藥。  \n2.若有使用軟便劑、利尿劑時，家屬應留意夜間孕產婦如廁需求。  \n3.若有暈眩、步態不穩、下肢無力者，暫勿下床。  \n四、如何預防跌倒？  \n待產時  \n1.為了監測胎兒的狀況，將會裝置胎兒監視器，裝置後仍可進行床上翻身、短暫坐起進食及四肢關節活\n動。 \n2.叫人鈴置放於待產婦可及之處，需要時可按鈴告知。  \n3.當需下床如廁時，務必有照顧者或護理 人員陪伴下，採漸進式下床，以免發生因頭暈或重心不穩而導致\n跌倒之危險。  \n4.若待產婦有施行減痛分娩，已經破水或產程進入活動期 (子宮頸擴張 ≧6公分)，則切勿下床，應使用床上\n便盆如廁。  \n產後照護  \n許多產婦在第一次下床時容易有暈眩感，尤其是在上完廁所要起身的時候。所以請務必注意 : \n1.產後第一次下床時都需家屬全程陪同。  \n2.採漸進式下床：先床頭搖高坐起 5分鐘；雙腳垂於床緣再坐 5分鐘；然後再站起來於床緣站 5分鐘，確\n認無頭暈現象，才能由家屬陪同至廁所。  \n3.預防低血糖 :除特殊情況需禁食外，可以攝取易消化之食物或溫熱飲食，以免低血糖發生頭暈情形。  \n4.感覺暈眩時，留在原地勿起身，請拉緊急叫人鈴。  \n5.移動身體動作勿太快。  \n防跌愛的小叮嚀' metadata={'source': './data/孕產婦住

In [ ]:
tools = load_tools(["human"], llm=llm, input_func=get_input)
tools += [
    childfall_qa_tool,
    pwomenfall_qa_tool,
]

agent_chain = initialize_agent(
    tools,
    llm,
    agent="chat-zero-shot-react-description", 
    verbose=True,
    handle_parsing_errors=True
)

In [9]:
agent_chain.run("有哪些跌倒原因？")

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I should check the Children fall QA system for information on the causes of falls in children.
Action: Children fall QA system
Action Input: Causes of falls in children

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and wil


Observation: Causes of falls in children can be categorized into different factors:

1. Child-related factors:
- Impaired consciousness, restlessness, weakness, or unsteady gait due to illness.
- High activity levels and a tendency to climb up and down.

2. Caregiver-related factors:
- Inadequate raising of both sides of the bed rails.
- Allowing the child to engage in activities on companion beds.
- Incorrect use of baby strollers or wheelchairs, including not using brakes or safety belts.
- Leaving the child unattended.

3. Medication-related factors:
- Sedatives or anesthesia causing drowsiness, dizziness, or lack of concentration.
- Diuretics or laxatives increasing the frequency of using the toilet.
- Hypoglycemic medications causing dizziness or low blood sugar.

4. Environmental factors:
- Slippery floors.
- Insufficient lighting.
- Inadequate availability of wheelchairs or strollers.

It is important to address these factors to prevent falls in children.


> Finished chain.


'Causes of falls in children can be categorized into different factors:\n\n1. Child-related factors:\n- Impaired consciousness, restlessness, weakness, or unsteady gait due to illness.\n- High activity levels and a tendency to climb up and down.\n\n2. Caregiver-related factors:\n- Inadequate raising of both sides of the bed rails.\n- Allowing the child to engage in activities on companion beds.\n- Incorrect use of baby strollers or wheelchairs, including not using brakes or safety belts.\n- Leaving the child unattended.\n\n3. Medication-related factors:\n- Sedatives or anesthesia causing drowsiness, dizziness, or lack of concentration.\n- Diuretics or laxatives increasing the frequency of using the toilet.\n- Hypoglycemic medications causing dizziness or low blood sugar.\n\n4. Environmental factors:\n- Slippery floors.\n- Insufficient lighting.\n- Inadequate availability of wheelchairs or strollers.\n\nIt is important to address these factors to prevent falls in children.'

In [35]:
agent_chain.run("跌倒怎麼辦？")

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Question: 跌倒怎麼辦？
Thought: This question seems to be asking for general advice on what to do when someone falls down.
Action:
```
{
  "action": "human",
  "action_input": "What should someone do when they fall down?"
}
```

What should someone do when they fall down?
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.

Observation: use your tools
Check your tools





Thought:

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Could not parse LLM output: I should ask a human for guidance on what to do when someone falls down.
Observation: Invalid or incomplete response
Thought:

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


...

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I should ask the Pregnant women fall QA system for guidance on what to do when a pregnant woman falls down.
Action:
```
{
  "action": "Pregnant women fall QA system",
  "action_input": "What should someone do when a pregnant woman falls down?"
}
```

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and wil


Observation: If a pregnant woman falls down, it is important to seek medical attention immediately. In addition, you should continue to observe the woman for any signs of the following:

1. Check for any external injuries such as fractures or swelling at the site of impact.
2. If the woman experiences severe abdominal or head pain.
3. If the woman feels regular contractions in the abdomen.
4. If there is continuous vaginal bleeding or a leakage of clear fluid (ruptured membranes).
5. If there is a decrease or absence of fetal movement.

If any of these symptoms occur, it is important to notify healthcare professionals.


> Finished chain.


'If a pregnant woman falls down, it is important to seek medical attention immediately. In addition, you should continue to observe the woman for any signs of the following:\n\n1. Check for any external injuries such as fractures or swelling at the site of impact.\n2. If the woman experiences severe abdominal or head pain.\n3. If the woman feels regular contractions in the abdomen.\n4. If there is continuous vaginal bleeding or a leakage of clear fluid (ruptured membranes).\n5. If there is a decrease or absence of fetal movement.\n\nIf any of these symptoms occur, it is important to notify healthcare professionals.'

In [16]:
agent_chain.run("跌倒常見的併發症有哪些？")

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Question: What are the common complications of falls?

Thought: I think I can use the Children fall QA system to find the answer to this question.

Action:
```
{
  "action": "Children fall QA system",
  "action_input": "What are the common complications of falls?"
}
```

/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/r07525009/.pyenv/versions/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and wil


Observation: The common complications of falls include local abrasions, bruises, bleeding, fractures in the impacted area, head injuries, and intracranial bleeding.


> Finished chain.


'The common complications of falls include local abrasions, bruises, bleeding, fractures in the impacted area, head injuries, and intracranial bleeding.'